In [1]:
'''To upload to github, all changes you make to a clone of this repo / notebook 
should be stored somewhere and you can just push in a terminal or github desktop'''

'To upload to github, all changes you make to a clone of this repo / notebook \nshould be stored somewhere and you can just push in a terminal or github desktop'

In [2]:
'''Everything here is made for the bag of words model he showed in Tuesday Nov 19 lecture 
- feel free to change anything, make your own model here or in a different ipynb

Essentially, the model takes all review data, splits it up into truncated words, 
and then makes a model using linear_model.Ridge that defines how much of an increase or decrease a word has
on the predicted outcome

nWords: # of words that the model takes into account to rate a book (currently 2000)

TODO: use n-grams
'''

'Everything here is made for the bag of words model he showed in Tuesday Nov 19 lecture \n- feel free to change anything, make your own model here or in a different ipynb\n\nEssentially, the model takes all review data, splits it up into truncated words, \nand then makes a model using linear_model.Ridge that defines how much of an increase or decrease a word has\non the predicted outcome\n\nnWords: # of words that the model takes into account to rate a book (currently 2000)\n\nTODO: use n-grams\n'

In [3]:
import gzip
import json
from tqdm import tqdm


# Path to the .json.gz file
file_path = 'goodreads_reviews_spoiler.json.gz'

# Open and read the file line by line
with gzip.open(file_path, 'rt', encoding='utf-8') as f:
    data = []
    for line in tqdm(f):
        # Parse each line as a separate JSON object
        try:
            review = json.loads(line)
            data.append(review)
        except json.JSONDecodeError as e:
            print(f"Error decoding line: {e}")

# Example: Print the first parsed review
if data:
    print(data[0])
else:
    print("No data was parsed.")


1378033it [00:34, 39675.58it/s]

{'user_id': '8842281e1d1347389f2ab93d60773d4d', 'timestamp': '2017-08-30', 'review_sentences': [[0, 'This is a special book.'], [0, 'It started slow for about the first third, then in the middle third it started to get interesting, then the last third blew my mind.'], [0, 'This is what I love about good science fiction - it pushes your thinking about where things can go.'], [0, "It is a 2015 Hugo winner, and translated from its original Chinese, which made it interesting in just a different way from most things I've read."], [0, 'For instance the intermixing of Chinese revolutionary history - how they kept accusing people of being "reactionaries", etc.'], [0, 'It is a book about science, and aliens.'], [0, 'The science described in the book is impressive - its a book grounded in physics and pretty accurate as far as I could tell.'], [1, 'Though when it got to folding protons into 8 dimensions I think he was just making stuff up - interesting to think about though.'], [1, 'But what woul

In [4]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
from tqdm import tqdm
import copy
from collections import Counter


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [5]:
dataset = []
for data_point in tqdm(data):
    user_id = data_point['user_id']
    book_id = data_point['book_id']
    has_spoiler = 1 if data_point['has_spoiler'] else 0
    dataset.append([user_id, book_id, has_spoiler])

100%|█████████████████████████████| 1378033/1378033 [00:01<00:00, 712078.87it/s]


In [6]:
import random

length = len(dataset)
def split_dataset(dataset, split_ratio=0.8):
    random.shuffle(dataset)
    
    split_index = int(len(dataset) * split_ratio)
    
    train_data = dataset[:split_index]
    test_data = dataset[split_index:]
    
    return train_data, test_data
trainset, testset = split_dataset(dataset)

In [7]:
dataset[0]

['fae84b8feeb9b3c6c2235d2570d1921b', '7802254', 0]

In [121]:
from collections import defaultdict

entriesPerUser = defaultdict(list)
entriesPerItem = defaultdict(list)
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)
for u,b,h in tqdm(trainset):
    if h == 1:
        entriesPerUser[u].append((b,h))
        entriesPerItem[b].append((u,h))
        usersPerItem[b].add(u)
        itemsPerUser[u].add(b)

100%|█████████████████████████████| 1102426/1102426 [00:03<00:00, 343810.19it/s]


In [15]:
def calcAccuracy(predictions, trueLabels):
    diff_arr = numpy.array(predictions) - numpy.array(trueLabels)
    correct_preds = 0
    for i in diff_arr:
        if i == 0:
            correct_preds += 1
    return correct_preds / len(trueLabels)

In [16]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [17]:
def JaccardItems(user, item):
    if user not in itemsPerUser or item not in usersPerItem:
        return 0
    books_user_has_read = itemsPerUser[user]
    users_who_read_item = usersPerItem[item]
    max_jaccard = 0
    for book in books_user_has_read:
        if item != book:
            users_who_read_book = usersPerItem[book]
            jaccard_sim = Jaccard(users_who_read_book, users_who_read_item)
            max_jaccard = max(max_jaccard, jaccard_sim)
    return max_jaccard

def JaccardUsers(user, item):
    if user not in itemsPerUser or item not in usersPerItem:
        return 0
    books_user_has_read = itemsPerUser[user]
    users_who_read_item = usersPerItem[item]
    max_jaccard = 0
    for reader in users_who_read_item:
        if user != reader:
            items_reader_has_read = itemsPerUser[reader]
            jaccard_sim = Jaccard(items_reader_has_read, books_user_has_read)
            max_jaccard = max(max_jaccard, jaccard_sim)         
    return max_jaccard

In [18]:
def Cosine(s1, s2):
    vec1 = Counter(s1)
    vec2 = Counter(s2)

    intersection = set(vec1.keys()) & set(vec2.keys())
    dot_product = sum(vec1[item] * vec2[item] for item in intersection)

    magnitude1 = math.sqrt(sum(count ** 2 for count in vec1.values()))
    magnitude2 = math.sqrt(sum(count ** 2 for count in vec2.values()))

    if magnitude1 == 0 or magnitude2 == 0:
        return 0

    return dot_product / (magnitude1 * magnitude2)

In [19]:
def CosineItems(user, item):
    if user not in itemsPerUser or item not in usersPerItem:
        return 0
    books_user_has_read = itemsPerUser[user]
    users_who_read_item = usersPerItem[item]
    max_cosine = 0
    for book in books_user_has_read:
        if item != book:
            users_who_read_book = usersPerItem[book]
            cosine_sim = Cosine(users_who_read_book, users_who_read_item)
            max_cosine = max(max_cosine, cosine_sim)
    return max_cosine

def CosineUsers(user, item):
    if user not in itemsPerUser or item not in usersPerItem:
        return 0
    books_user_has_read = itemsPerUser[user]
    users_who_read_item = usersPerItem[item]
    max_cosine = 0
    for reader in users_who_read_item:
        if user != reader:
            items_reader_has_read = itemsPerUser[reader]
            cosine_sim = Cosine(items_reader_has_read, books_user_has_read)
            max_cosine = max(max_cosine, cosine_sim)         
    return max_cosine

In [20]:
from collections import Counter
def Pearson(s1, s2):
    vec1 = Counter(s1)
    vec2 = Counter(s2)

    intersection = set(vec1.keys()) & set(vec2.keys())
    if not intersection:
        return 0 

    mean1 = sum(vec1[item] for item in intersection) / len(intersection)
    mean2 = sum(vec2[item] for item in intersection) / len(intersection)

    numer = sum((vec1[item] - mean1) * (vec2[item] - mean2) for item in intersection)
    denom1 = sum((vec1[item] - mean1) ** 2 for item in intersection)
    denom2 = sum((vec2[item] - mean2) ** 2 for item in intersection)

    denom = math.sqrt(denom1) * math.sqrt(denom2)
    if denom == 0:
        return 0

    return numer / denom

In [21]:
def PearsonItems(user, item):
    if user not in itemsPerUser or item not in usersPerItem:
        return 0
    books_user_has_read = itemsPerUser[user]
    users_who_read_item = usersPerItem[item]
    max_pearson = 0
    for book in books_user_has_read:
        if item != book:
            users_who_read_book = usersPerItem[book]
            pearson_sim = Pearson(users_who_read_book, users_who_read_item)
            max_pearson = max(max_pearson, pearson_sim)
    return max_pearson

def PearsonUsers(user, item):
    if user not in itemsPerUser or item not in usersPerItem:
        return 0
    books_user_has_read = itemsPerUser[user]
    users_who_read_item = usersPerItem[item]
    max_pearson = 0
    for reader in users_who_read_item:
        if user != reader:
            items_reader_has_read = itemsPerUser[reader]
            pearson_sim = Pearson(items_reader_has_read, books_user_has_read)
            max_pearson = max(max_pearson, pearson_sim)         
    return max_pearson

In [22]:
def feat(u, b, h):
    return [1] + [JaccardItems(u, b), JaccardUsers(u, b)] + [CosineItems(u, b), CosineUsers(u, b)] + [PearsonItems(u, b), PearsonUsers(u, b)]      

In [130]:
Xtrain = []
for u, b, h in tqdm(trainset):
    Xtrain.append(feat(u, b, h))

100%|███████████████████████████████| 1102426/1102426 [05:26<00:00, 3381.33it/s]


In [131]:
ytrain = [r for _, _, r in tqdm(trainset)]
Xtest = [feat(u, b, r) for u, b, r in tqdm(testset)]
ytest = [r for _, _, r in tqdm(testset)]

100%|███████████████████████████████| 275607/275607 [00:01<00:00, 262735.10it/s]


In [132]:
true_labels = []
for u, b, h in testset:
    true_labels.append(h)

In [138]:
C_arr = [1000, 100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001]
for c in C_arr:
    print("Model C = " + str(c))
    mod = linear_model.LogisticRegression(class_weight='balanced', C=c)
    mod.fit(Xtrain, ytrain)
    test_pred = (mod.predict_proba(Xtest)[:, 1] >= 1).astype(int)
    print(calcAccuracy(test_pred, true_labels))
    diff_arr = numpy.array(test_pred) - numpy.array(true_labels)
    sum_arr = numpy.array(test_pred) + numpy.array(true_labels)
    FP, FN = 0, 0
    TP, TN = 0, 0
    for i in diff_arr:
        if i == 1:
            FP += 1
        if i == -1:
            FN += 1
    for i in sum_arr:
        if i==0:
            TN += 1
        if i == 2:
            TP += 1
    print("FP: " + str(FP))
    print("FN: " + str(FN))
    print("TP: " + str(TP))
    print("TN: " + str(TN))
    print("\n")

Model C = 1000
0.9349109420297743
FP: 0
FN: 17939
TP: 0
TN: 257668


Model C = 100
0.9349109420297743
FP: 0
FN: 17939
TP: 0
TN: 257668


Model C = 10
0.9349109420297743
FP: 0
FN: 17939
TP: 0
TN: 257668


Model C = 1
0.9349109420297743
FP: 0
FN: 17939
TP: 0
TN: 257668


Model C = 0.1
0.9349109420297743
FP: 0
FN: 17939
TP: 0
TN: 257668


Model C = 0.01
0.9349109420297743
FP: 0
FN: 17939
TP: 0
TN: 257668


Model C = 0.001
0.9349109420297743
FP: 0
FN: 17939
TP: 0
TN: 257668


Model C = 0.0001
0.9349109420297743
FP: 0
FN: 17939
TP: 0
TN: 257668


Model C = 1e-05
0.9349109420297743
FP: 0
FN: 17939
TP: 0
TN: 257668




In [141]:
from sklearn.svm import LinearSVC

C_arr = [1000, 100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001]

for c in C_arr:
    model = LinearSVC(random_state=42, max_iter=100000, C=c)
    model.fit(Xtrain, ytrain)
    y_pred = model.predict(Xtest)
    accuracy = calcAccuracy(y_pred, true_labels)
    print("C = " + str(c) + ", " + "Accuracy: " + str(accuracy))
    # diff_arr = numpy.array(test_pred) - numpy.array(true_labels)
    # sum_arr = numpy.array(test_pred) + numpy.array(true_labels)
    # FP, FN = 0, 0
    # TP, TN = 0, 0
    # for i in diff_arr:
    #     if i == 1:
    #         FP += 1
    #     if i == -1:
    #         FN += 1
    # for i in sum_arr:
    #     if i==0:
    #         TN += 1
    #     if i == 2:
    #         TP += 1
    # print("FP: " + str(FP))
    # print("FN: " + str(FN))
    # print("TP: " + str(TP))
    # print("TN: " + str(TN))
    # print("\n")

C = 1000, Accuracy: 0.9270664388059809
C = 100, Accuracy: 0.9270664388059809
C = 10, Accuracy: 0.9270664388059809
C = 1, Accuracy: 0.9270628104511134
C = 0.1, Accuracy: 0.9271244924838629
C = 0.01, Accuracy: 0.9275925502617858
C = 0.001, Accuracy: 0.9288733595300555
C = 0.0001, Accuracy: 0.9321098520719721
C = 1e-05, Accuracy: 0.9349109420297743


In [107]:
# Under-Sampling Data

In [78]:
spoiler_reviews = [entry for entry in trainset if entry[2] == 1]
non_spoiler_reviews = [entry for entry in trainset if entry[2] == 0]

sampled_non_spoiler_reviews = random.sample(non_spoiler_reviews, len(spoiler_reviews))
balanced_trainset = spoiler_reviews + sampled_non_spoiler_reviews
len(balanced_trainset)
# Not feasible because our train set ends up being smaller than the test set

143948

In [108]:
# Synthetic Data Generation

In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [46]:
trainset = np.array(trainset)
user_encoder = LabelEncoder()
book_encoder = LabelEncoder()

user_encoded = user_encoder.fit_transform(trainset[:, 0])
book_encoded = book_encoder.fit_transform(trainset[:, 1])
X = np.column_stack((user_encoded, book_encoded))
y = trainset[:, 2].astype(int)

smote = SMOTE(sampling_strategy='auto', random_state=42)

X_resampled, y_resampled = smote.fit_resample(X, y)

user_resampled = user_encoder.inverse_transform(X_resampled[:, 0].astype(int))
book_resampled = book_encoder.inverse_transform(X_resampled[:, 1].astype(int))

resampled_trainset = np.column_stack((user_resampled, book_resampled, y_resampled))

In [47]:
resampled_trainset = [[i[0], i[1], 1 if i[2] == "1" else 0] for i in tqdm(resampled_trainset)]

100%|█████████████████████████████| 2061832/2061832 [00:02<00:00, 974607.56it/s]


In [48]:
resampled_trainset[0]
len(resampled_trainset)

2061832

In [49]:
np.random.shuffle(resampled_trainset)
resampled_trainset = resampled_trainset[:len(resampled_trainset) // 2]
# resampled_trainset[0]
len(resampled_trainset)

1030916

In [50]:
from collections import defaultdict

entriesPerUser = defaultdict(list)
entriesPerItem = defaultdict(list)
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)
for u,b,h in tqdm(resampled_trainset):
    if h == 1:
        entriesPerUser[u].append((b,h))
        entriesPerItem[b].append((u,h))
        usersPerItem[b].add(u)
        itemsPerUser[u].add(b)

100%|█████████████████████████████| 1030916/1030916 [00:01<00:00, 663721.01it/s]


In [51]:
Xtrain = []
for u, b, h in tqdm(resampled_trainset):
    Xtrain.append(feat(u, b, h))

ytrain = [r for _, _, r in tqdm(resampled_trainset)]

Xtest = [feat(u, b, r) for u, b, r in tqdm(testset)]

ytest = [r for _, _, r in tqdm(testset)]

100%|███████████████████████████████| 275607/275607 [00:01<00:00, 216443.14it/s]


In [52]:
true_labels = []
for u, b, h in testset:
    true_labels.append(h)

In [53]:
C_arr = [1000, 100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001]
for c in C_arr:
    print("Model C = " + str(c))
    mod = linear_model.LogisticRegression(class_weight='balanced', C=c)
    mod.fit(Xtrain, ytrain)
    test_pred = (mod.predict_proba(Xtest)[:, 1] >= 1).astype(int)
    print(calcAccuracy(test_pred, true_labels))
    diff_arr = numpy.array(test_pred) - numpy.array(true_labels)
    sum_arr = numpy.array(test_pred) + numpy.array(true_labels)
    FP, FN = 0, 0
    TP, TN = 0, 0
    for i in diff_arr:
        if i == 1:
            FP += 1
        if i == -1:
            FN += 1
    for i in sum_arr:
        if i==0:
            TN += 1
        if i == 2:
            TP += 1
    print("FP: " + str(FP))
    print("FN: " + str(FN))
    print("TP: " + str(TP))
    print("TN: " + str(TN))
    print("\n")

Model C = 1000
0.934265094863338
FP: 0
FN: 18117
TP: 0
TN: 257490


Model C = 100
0.934265094863338
FP: 0
FN: 18117
TP: 0
TN: 257490


Model C = 10
0.934265094863338
FP: 0
FN: 18117
TP: 0
TN: 257490


Model C = 1
0.934265094863338
FP: 0
FN: 18117
TP: 0
TN: 257490


Model C = 0.1
0.934265094863338
FP: 0
FN: 18117
TP: 0
TN: 257490


Model C = 0.01
0.934265094863338
FP: 0
FN: 18117
TP: 0
TN: 257490


Model C = 0.001
0.934265094863338
FP: 0
FN: 18117
TP: 0
TN: 257490


Model C = 0.0001
0.934265094863338
FP: 0
FN: 18117
TP: 0
TN: 257490


Model C = 1e-05
0.934265094863338
FP: 0
FN: 18117
TP: 0
TN: 257490




In [54]:
from sklearn.svm import LinearSVC

C_arr = [1000, 100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001]

for c in C_arr:
    model = LinearSVC(random_state=42, max_iter=100000, C=c)
    model.fit(Xtrain, ytrain)
    y_pred = model.predict(Xtest)
    accuracy = calcAccuracy(y_pred, true_labels)
    print("C = " + str(c) + ", " + "Accuracy: " + str(accuracy))
    diff_arr = numpy.array(test_pred) - numpy.array(true_labels)
    sum_arr = numpy.array(test_pred) + numpy.array(true_labels)
    FP, FN = 0, 0
    TP, TN = 0, 0
    for i in diff_arr:
        if i == 1:
            FP += 1
        if i == -1:
            FN += 1
    for i in sum_arr:
        if i==0:
            TN += 1
        if i == 2:
            TP += 1
    print("FP: " + str(FP))
    print("FN: " + str(FN))
    print("TP: " + str(TP))
    print("TN: " + str(TN))
    print("\n")

C = 1000, Accuracy: 0.8905434187085234
FP: 0
FN: 18117
TP: 0
TN: 257490


C = 100, Accuracy: 0.8905361619987882
FP: 0
FN: 18117
TP: 0
TN: 257490


C = 10, Accuracy: 0.8905724455474643
FP: 0
FN: 18117
TP: 0
TN: 257490


C = 1, Accuracy: 0.8906123574510081
FP: 0
FN: 18117
TP: 0
TN: 257490


C = 0.1, Accuracy: 0.889574647958869
FP: 0
FN: 18117
TP: 0
TN: 257490


C = 0.01, Accuracy: 0.8872488724887249
FP: 0
FN: 18117
TP: 0
TN: 257490


C = 0.001, Accuracy: 0.8818680222200452
FP: 0
FN: 18117
TP: 0
TN: 257490


C = 0.0001, Accuracy: 0.873403070313889
FP: 0
FN: 18117
TP: 0
TN: 257490


C = 1e-05, Accuracy: 0.8610666637639828
FP: 0
FN: 18117
TP: 0
TN: 257490




In [55]:
# Stratified Sampling

In [56]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(dataset, test_size=0.2, stratify=[entry[2] for entry in dataset])

In [58]:
len(test_set)

275607

In [59]:
from collections import defaultdict

entriesPerUser = defaultdict(list)
entriesPerItem = defaultdict(list)
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)
for u,b,h in tqdm(trainset):
    if h == 1:
        entriesPerUser[u].append((b,h))
        entriesPerItem[b].append((u,h))
        usersPerItem[b].add(u)
        itemsPerUser[u].add(b)

100%|█████████████████████████████| 1102426/1102426 [00:01<00:00, 724579.60it/s]


In [60]:
Xtrain = []
for u, b, h in tqdm(trainset):
    Xtrain.append(feat(u, b, h))

ytrain = [r for _, _, r in tqdm(trainset)]

Xtest = [feat(u, b, r) for u, b, r in tqdm(testset)]

ytest = [r for _, _, r in tqdm(testset)]

100%|██████████████████████████████| 275607/275607 [00:00<00:00, 1518564.04it/s]


In [61]:
C_arr = [1000, 100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001]
for c in C_arr:
    print("Model C = " + str(c))
    mod = linear_model.LogisticRegression(class_weight='balanced', C=c)
    mod.fit(Xtrain, ytrain)
    test_pred = (mod.predict_proba(Xtest)[:, 1] >= 1).astype(int)
    print(calcAccuracy(test_pred, true_labels))
    diff_arr = numpy.array(test_pred) - numpy.array(true_labels)
    sum_arr = numpy.array(test_pred) + numpy.array(true_labels)
    FP, FN = 0, 0
    TP, TN = 0, 0
    for i in diff_arr:
        if i == 1:
            FP += 1
        if i == -1:
            FN += 1
    for i in sum_arr:
        if i==0:
            TN += 1
        if i == 2:
            TP += 1
    print("FP: " + str(FP))
    print("FN: " + str(FN))
    print("TP: " + str(TP))
    print("TN: " + str(TN))
    print("\n")

Model C = 1000
0.934265094863338
FP: 0
FN: 18117
TP: 0
TN: 257490


Model C = 100
0.934265094863338
FP: 0
FN: 18117
TP: 0
TN: 257490


Model C = 10
0.934265094863338
FP: 0
FN: 18117
TP: 0
TN: 257490


Model C = 1
0.934265094863338
FP: 0
FN: 18117
TP: 0
TN: 257490


Model C = 0.1
0.934265094863338
FP: 0
FN: 18117
TP: 0
TN: 257490


Model C = 0.01
0.934265094863338
FP: 0
FN: 18117
TP: 0
TN: 257490


Model C = 0.001
0.934265094863338
FP: 0
FN: 18117
TP: 0
TN: 257490


Model C = 0.0001
0.934265094863338
FP: 0
FN: 18117
TP: 0
TN: 257490


Model C = 1e-05
0.934265094863338
FP: 0
FN: 18117
TP: 0
TN: 257490


